# Data Engineering 1: Lab 06
---------------

### Task 01: Consistent Hashing
#### Fill the parts marked with Todo for a complete implementation of Consistent Hashing

In [209]:
import names
import pandas as pd


#returns list of size n with random names
def getData(n):
    data = []
    i = 0
    while i < n:
        data.append(names.get_first_name())
        i += 1
    return data

#returns list of hash values for list of data
def getHashs(data):
    hashs = []
    for word in data:
        hashs.append(abs(hash(word)) % (10 ** 8))
    return hashs

#returns list of size m with random server names
def getServers(m):
    data = []
    i = 0
    while i < m:
        data.append('Server_' + names.get_first_name())
        i += 1
    return data

def getAngles(hashs):
    angle = [(360 / (10 ** 8)) * hash_value for hash_value in hashs]
    return angle


#TodO: create a DataFrame with the list created with the function getdata(100) 
# and a type column filled with the type key

df_names = pd.DataFrame(getData(100))
df_names.rename(columns={0:'name'}, inplace=True)
df_names['type'] = 'key'
    
#Todo: add a column to the DataFrame with the hashs created with the getHashs() method

hashs = getHashs(df_names['name'])
df_names['hash'] = hashs

#Todo: create m servers with the getServers function and add them to the DataFrame with the type server

df_servers = pd.DataFrame(getServers(10))
df_servers.rename(columns={0:'name'}, inplace=True)
df_servers['type'] = 'server'
hashs = getHashs(df_servers['name'])
df_servers['hash'] = hashs

df = pd.merge(df_names, df_servers, how='outer')


#Todo: add a column to the DataFrame with the angle on the circle

df['angle'] = getAngles(hashs)
df

#Todo: add a column to the DataFrame with the corresponding server

#Todo: print the final DataFrame

ValueError: Length of values (10) does not match length of index (100)

### Task 02: Balanced Consistent Hashing
#### Extend the code from Task 01 to ensure object keys are evenly distributed among servers

### Task 03: Weighted Balanced Consistent Hashing
#### Extend the code from Task 01 to ensure object keys are evenly distributed among servers but also with a weighting of the single servers.